In [15]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import legacy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import legacy
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [20]:
def normalize(df):
    
    for column in ['x', 'y', 'z']:
        df[column] = (df[column] - df[column].mean()) / df[column].std()
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

def segment(df, window_size, step_size):
    segments = []
    for start in range(0, len(df) - window_size + 1, step_size):
        end = start + window_size

        \
        segment = df.iloc[start:end]
        segments.append(segment)
    return segments

In [21]:
def segment_activities(window_size=32, step_size=16):
    # window_size = 160  # 2.5 seconds
    # step_size = 20  # 75% overlap

    activities_path = 'Activity'
    activity_segments = {}

    # Iterate over each activity's folder
    for activity_name in os.listdir(activities_path):
        print(activity_name)
        activity_folder = os.path.join(activities_path, activity_name)
        if os.path.isdir(activity_folder):
            # Store segments for each activity
            activity_segments[activity_name] = []
            
            # Iterate over each CSV file within the activity's folder
            for filename in os.listdir(activity_folder):
                if filename.endswith('.csv'):
                    file_path = os.path.join(activity_folder, filename)
                    
                    # Read  CSV file
                    df = pd.read_csv(file_path)
                    
                    # Normalize data
                    df_normalized = normalize(df)
                    
                    # Segment the data using a rolling window
                    segments = segment(df_normalized, window_size, step_size)
                    
                    # Append the segments to the activity's list
                    activity_segments[activity_name].extend(segments)
    return activity_segments


In [22]:
def fit_model(activity_segments):
    X, y = [], []


    # Convert the segments into a suitable format for training
    for activity_name, segments in activity_segments.items():
        for segment in segments:
            X.append(segment.to_numpy())  # Assuming segment is a pandas DataFrame
            y.append(activity_name)
    X = np.array(X)
    y = np.array(y)
    print("segments in suitable format")


    # Encode the activity labels into integers
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print("labels encoded")
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    # Define the LSTM model
    model = Sequential([
        LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
        LSTM(64),
        Dense(len(le.classes_), activation='softmax')
    ])
    print("comiling model")
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print("training model")
    # Train the model
    model.fit(X_train, y_train, epochs=10, validation_split=0.2)

    # Evaluate the model
    model.evaluate(X_test, y_test)
    return model

# 5 Second Window, 1 Second Overlap Data

In [6]:
##new Method
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X, y = [], []


# Convert the segments into a suitable format for training
for activity_name, segments in activity_segments.items():
    for segment in segments:
        X.append(segment.to_numpy())  # Assuming segment is a pandas DataFrame
        y.append(activity_name)
X = np.array(X)
y = np.array(y)
print("segments in suitable format")


# Encode the activity labels into integers
le = LabelEncoder()
y_encoded = le.fit_transform(y)
print("labels encoded")
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    LSTM(64),
    Dense(len(le.classes_), activation='softmax')
])
print("comiling model")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("training model")
# Train the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the model
model.evaluate(X_test, y_test)



segments in suitable format
segments encoded
comiling model
training model
Epoch 1/10
13943/13943 [==============================] - 1589s 114ms/step - loss: 2.0645 - accuracy: 0.3276 - val_loss: 1.5044 - val_accuracy: 0.4773
Epoch 2/10
13943/13943 [==============================] - 1532s 110ms/step - loss: 1.2374 - accuracy: 0.5534 - val_loss: 1.0387 - val_accuracy: 0.6173
Epoch 3/10
13943/13943 [==============================] - 1569s 113ms/step - loss: 0.9156 - accuracy: 0.6645 - val_loss: 0.8274 - val_accuracy: 0.6926
Epoch 4/10
13943/13943 [==============================] - 1534s 110ms/step - loss: 0.7488 - accuracy: 0.7198 - val_loss: 0.6891 - val_accuracy: 0.7434
Epoch 5/10
13943/13943 [==============================] - 8596s 617ms/step - loss: 0.6303 - accuracy: 0.7621 - val_loss: 0.6068 - val_accuracy: 0.7726
Epoch 6/10
13943/13943 [==============================] - 1567s 112ms/step - loss: 0.5448 - accuracy: 0.7908 - val_loss: 0.5200 - val_accuracy: 0.8022
Epoch 7/10
13943/13

[0.4904477298259735, 0.8195797204971313]

# FFT Data 5 Second Window, 1 Second Overlap

In [8]:
import numpy as np
from numpy.fft import fft
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def apply_fft(segment):
    # Apply FFT and calculate the magnitude
    fft_values = fft(segment)
    magnitude = np.abs(fft_values)
    return magnitude

# Assuming 'activity_segments' is a dictionary with the structure mentioned earlier
X_fft = []  # FFT features
y = []  # Labels

for activity_name, segments in activity_segments.items():
    for segment in segments:
        # Assuming each segment is a pandas DataFrame with 'x', 'y', 'z' columns
        segment_fft = np.vstack([apply_fft(segment[col]) for col in ['x', 'y', 'z']]).T  # Apply FFT to each axis and transpose
        X_fft.append(segment_fft)
        y.append(activity_name)

X_fft = np.array(X_fft, dtype=object)
y = np.array(y)
print("segments in suitable format\nApplied FFT")

# Encode the activity labels into integers
le = LabelEncoder()
y_encoded = le.fit_transform(y)
print("encoded labels")
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_fft, y_encoded, test_size=0.2, random_state=42)

# Adjusting input shapes for LSTM based on FFT features
input_shape = (X_train[0].shape[0], X_train[0].shape[1])

# Define the LSTM model
model = Sequential([
    LSTM(128, input_shape=input_shape, return_sequences=True),
    LSTM(64),
    Dense(len(le.classes_), activation='softmax')
])
print("compiled model\nTraining Model")

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Since the datasets can have variable lengths due to FFT, use a generator or pad sequences for batching
# Example code for model training and evaluation is omitted due to complexity around handling variable input lengths



segments in suitable format
Applied FFT
encoded labels
compiled model
Training Model


In [9]:
# Pad sequences to ensure uniform input size for LSTM
# Find the longest sequence
max_length = max(len(sample) for sample in X_fft)

# Pad all sequences to the length of the longest sequence
X_fft_padded = pad_sequences(X_fft, maxlen=max_length, dtype='float32', padding='post')
print("padded the data")
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_fft_padded, y_encoded, test_size=0.2, random_state=42)
print("training and testing data")
# Adjusting input shapes for LSTM based on FFT features
input_shape = (X_train.shape[1], X_train.shape[2])

# Define the LSTM model
model = Sequential([
    LSTM(128, input_shape=input_shape, return_sequences=True),
    LSTM(64),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("compiled model")
# Train the model
print("training model")
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

padded the data
training and testing data
compiled model
training model
Epoch 1/10
13943/13943 [==============================] - 1511s 108ms/step - loss: 1.8652 - accuracy: 0.3859 - val_loss: 1.6013 - val_accuracy: 0.4551
Epoch 2/10
13943/13943 [==============================] - 1526s 109ms/step - loss: 1.4807 - accuracy: 0.4891 - val_loss: 1.3956 - val_accuracy: 0.5121
Epoch 3/10
13943/13943 [==============================] - 1439s 103ms/step - loss: 1.2980 - accuracy: 0.5393 - val_loss: 1.2635 - val_accuracy: 0.5506
Epoch 4/10
13943/13943 [==============================] - 1598s 115ms/step - loss: 1.1838 - accuracy: 0.5724 - val_loss: 1.1821 - val_accuracy: 0.5701
Epoch 5/10
13943/13943 [==============================] - 1576s 113ms/step - loss: 1.1060 - accuracy: 0.5946 - val_loss: 1.1134 - val_accuracy: 0.5917
Epoch 6/10
13943/13943 [==============================] - 1494s 107ms/step - loss: 1.0490 - accuracy: 0.6104 - val_loss: 1.0971 - val_accuracy: 0.5957
Epoch 7/10
13943/13943

# 3 Second Window 3/2 Second Overlap

In [27]:
activity_segments = segment_activities(96, 12)
model = fit_model(activity_segments)

Motor Behavioral Task: (All conditions seen here)
Motor Behavioral
Training ITWT - A
digit symbol task: (SIT)
TWT_B training
Training:TWT_B
B1_T2: (WALK)
trail making task: (SIT)
Naughton Test: (WALK)
B1_T1: (WALK)
Training: TWT_A
Motor behavioral task
Training ITWT -B
TNT
TWT_A Training: (WALK)
TM Comfortable Speed: (WALK)
Naughton test
B2_T1: (WALK)
B1_TWT_A: (WALK)
TWT_A training
Training ITWT - B
Training:TWT_A
TWT_B Training: (WALK)
Naughton
B2_TWT_A: (WALK)
SOT: (STAND)
Montreal Cognitive Assessment: (SIT)
HR Recovery: (STAND or SIT)
B1_TWT_B: (WALK)
B2_T2: (WALK)
Naughton Task
TM comfortable speed
Training: TWT_B
B2_TWT_B: (WALK)
segments in suitable format
labels encoded
comiling model
training model
Epoch 1/10
23335/23335 [==============================] - 1324s 57ms/step - loss: 1.8501 - accuracy: 0.3872 - val_loss: 1.3316 - val_accuracy: 0.5344
Epoch 2/10
23335/23335 [==============================] - 4629s 198ms/step - loss: 1.1433 - accuracy: 0.5951 - val_loss: 0.9908 - va

# 1 Second Window 1/2 Second Overlap

In [23]:
activity_segments = segment_activities()
model = fit_model(activity_segments)

Motor Behavioral Task: (All conditions seen here)
Motor Behavioral
Training ITWT - A
digit symbol task: (SIT)
TWT_B training
Training:TWT_B
B1_T2: (WALK)
trail making task: (SIT)
Naughton Test: (WALK)
B1_T1: (WALK)
Training: TWT_A
Motor behavioral task
Training ITWT -B
TNT
TWT_A Training: (WALK)
TM Comfortable Speed: (WALK)
Naughton test
B2_T1: (WALK)
B1_TWT_A: (WALK)
TWT_A training
Training ITWT - B
Training:TWT_A
TWT_B Training: (WALK)
Naughton
B2_TWT_A: (WALK)
SOT: (STAND)
Montreal Cognitive Assessment: (SIT)
HR Recovery: (STAND or SIT)
B1_TWT_B: (WALK)
B2_T2: (WALK)
Naughton Task
TM comfortable speed
Training: TWT_B
B2_TWT_B: (WALK)
segments in suitable format
labels encoded
comiling model
training model
Epoch 1/10
17589/17589 [==============================] - 370s 21ms/step - loss: 1.7460 - accuracy: 0.4177 - val_loss: 1.4398 - val_accuracy: 0.5023
Epoch 2/10
17589/17589 [==============================] - 368s 21ms/step - loss: 1.3162 - accuracy: 0.5432 - val_loss: 1.2408 - val_a